# EEG Viewer
![status](https://img.shields.io/badge/status-in%20progress-orange)



<div style="text-align: center;">
    <img src="./assets/230524_eeg-viewer.png" alt="eeg viewer preview" width="450"/>
</div>

## Summary

This workflow is intended to demonstrate the visualization of a set of 1D EEG timeseries with HoloViz and Bokeh tools.

For details specific to this workflow, such as goals, specifications, and bottlenecks, please see this workflow's [readme](./readme_eeg-viewer.md).

For a summary of EEG research, data, and software, please see [neuro/wiki/EEG-notes](https://github.com/holoviz-topics/neuro/wiki/EEG-notes).

## Imports and config

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Requirements</p>
    <p>This workflow notebook requires the <a href="./environment.yml">environment</a> specified in this workflow directory.</p>
</div>


In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np

import holoviews as hv; hv.extension('bokeh')
from holoviews.plotting.links import RangeToolLink
from holoviews.operation.datashader import rasterize
from holoviews import Dataset

from neurodatagen.eeg import generate_eeg_powerlaw
from hvneuro import download_file

from bokeh.models import HoverTool
from scipy.stats import zscore
import panel as pn; pn.extension(template='material')

## Generate simulated data

The `generate_eeg_powerlaw` function synthesizes EEG data as high-pass filtered pink noise power law time series by default. The function returns a 2D numpy array of synthetic EEG data (in microvolts) shaped as (number of channels, total samples), a 1D time array (in seconds), and a list of channel names. Parameters such as the high-pass filter factor (in Hz) and an amplitude scaling factor allow customization of the generated data.

In [ ]:
n_channels = 25
n_seconds = 30
fs = 512

data, time, channels = generate_eeg_powerlaw(n_channels, n_seconds, fs)

print(f'shape: {data.shape} (n_channels, samples) ')
data

## Visualize synthetic EEG data with minimap

In [ ]:
max_ch_disp = 10  # max channels to initially display
max_t_disp = 3 # max time in seconds to initially display

spacing = 5.5  # Spacing between channels
offset = np.std(data) * spacing

# Create a hv.Curve element per chan.
# Note: alternative is to call hv.Path once on offset-adjusted data, but 
# then we couldn't independently apply formating to the channels (which 
# we aren't doing yet, but we likely will in the future)
channel_curves = []
max_data = data.max()
hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "@original_amplitude µV")])
for i, channel_data in enumerate(data):
    offset_data = channel_data + (i * offset)
    max_data = max(offset_data.max(), max_data) # update max
    ds = Dataset((time, offset_data, channel_data, channels[i]), ["Time", "Amplitude", "original_amplitude", "channel"])
    channel_curves.append(
        hv.Curve(ds, "Time", ["Amplitude", "original_amplitude", "channel"]).opts(
            color="black", line_width=1,
            tools=[hover, 'xwheel_zoom'], shared_axes=False))

# Create mapping from yaxis location to ytick for each channel
# so we can have categorical-style labeling on a continuous axis.
# Note: this would/should change when we implement independent 
# coordinates.
yticks = [(i * offset, ich) for i, ich in enumerate(channels)]


# Create an overlay of curves
# TODO.. setting x/y_range bounds does not yet restrict the RangeTool from going beyond these limits
# TODO.. the zoom out will stop when it hits any single bound, and not continue zooming out in other directions/dims
eeg_viewer = hv.Overlay(channel_curves, kdims="Channel").opts(
    padding=0, xlabel="Time (s)", ylabel="Channel", #default_tools=['hover', 'pan', 'box_zoom', 'save', 'reset'],
    yticks=yticks, show_legend=False, aspect=1.5, responsive=True,
    shared_axes=False, backend_opts={
        "x_range.bounds": (time.min(), time.max()),
        "y_range.bounds": (data.min(), max_data)})

# Get the y positions of the yticks to use as yaxis of minimap image
y_positions, _ = zip(*yticks)

# Compute z-scores across time for each channel
z_data = zscore(data, axis=1)

# Generate the zscored image for the minimap using the y tiack positions from the eeg_viewer
minimap = hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)")

# RangeTool doesn't work with rasterized object? TODO: file issue
# minimap = rasterize(minimap)

# maybe I should datashade/2d-bin the data before creating the hv.Image
# I could use lttb (1d so per channel) or ResampleOperation2D (but I think that applies to the entire nb)
# or some operation from datashader to return the 2d hist


# Style the minimap 
minimap = minimap.opts(
    cmap="RdBu_r", colorbar=False, xlabel='', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=100, responsive=True, default_tools=[''], shared_axes=False, clim=(-z_data.std()*2.5, z_data.std()*2.5))
    
# Create RangeToolLink between the minimap and the main EEG viewer 
# (quirk: apply to just one eeg trace and it will apply to all. see HoloViews #4472)
max_y_disp = np.max(data[max_ch_disp-1,:] + ((max_ch_disp-1) * offset))
RangeToolLink(minimap, list(eeg_viewer.values())[0], axes=["x", "y"],
              boundsx=(None, max_t_disp),
              boundsy=(None, max_y_disp))

# Display vertically
# layout = (eeg_viewer + minimap).cols(1).opts(shared_axes=False, merge_tools=False)
# eeg_app = pn.Row(layout).servable() # too much spacing between plots in served app
eeg_app = pn.Column(pn.Row(eeg_viewer, min_height=500, sizing_mode='stretch_both'), minimap, sizing_mode='stretch_both')#.servable()#target='main') # BUG Panel #5315: rangetool is variably active in the bokeh toolbar on eeg viewer plot.. not respecting shared_axes=False
eeg_app

## Load and plot real data

In [ ]:
import mne

In [ ]:
# This dataset is 2.6 MB on disk
url = "https://physionet.org/files/eegmmidb/1.0.0/S001/S001R04.edf?download"
local_data_path = "../../data/"

# Will not download if already present at local_data_path
local_file_path = download_file(url, local_data_path)

In [ ]:
raw = mne.io.read_raw_edf(local_file_path, preload=True)

In [ ]:
raw.info

In [ ]:
# preview the channel names, types, signal ranges, and uncompressed size
raw.describe()

In [ ]:
# clean up the channel names
raw.rename_channels(lambda s: s.strip("."));

In [ ]:
# preview available montages that are shipped with MNE
# mne.channels.get_builtin_montages(descriptions=True)

In [ ]:
# Let's use the standard 10-20
montage = mne.channels.make_standard_montage("standard_1020")

In [ ]:
# plot the assigned positions of our data channels
raw.set_montage(montage, match_case=False)
sphere=(0, 0.015, 0, 0.099) #manually adjust the y origin coord and radius a bit
raw.plot_sensors(show_names=True, sphere=sphere);

In [ ]:
# re-reference EEG data to the average over all recording channels
raw.set_eeg_reference("average");

In [ ]:
time = raw.times
channels = raw.ch_names

# get the EEG data (for this data set, all channels are EEG anyways)
eeg_indices = mne.pick_types(raw.info, eeg=True)
data = raw.get_data(picks=eeg_indices, units={"eeg":"uV"})

In [ ]:
max_ch_disp = 10  # max channels to initially display
max_t_disp = 5 # max time in seconds to initially display

spacing = 2.5  # Spacing between channels
offset = np.std(data) * spacing

# Create a hv.Curve element per channel
channel_curves = []
max_data = data.max()
hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "@original_amplitude µV")])
for i, channel_data in enumerate(data):
    offset_data = channel_data + (i * offset)
    max_data = max(offset_data.max(), max_data)  # update max
    ds = Dataset((time, offset_data, channel_data, channels[i]), ["Time", "Amplitude", "original_amplitude", "channel"])
    channel_curves.append(
        hv.Curve(ds, "Time", ["Amplitude", "original_amplitude", "channel"]).opts(
            color="black", line_width=1,
            tools=[hover, 'xwheel_zoom'], shared_axes=False))

# Create mapping from yaxis location to ytick for each channel
yticks = [(i * offset, ich) for i, ich in enumerate(channels)]

# Create an overlay of curves
eeg_viewer = hv.Overlay(channel_curves, kdims="Channel").opts(
    padding=0, xlabel="Time (s)", ylabel="Channel", yticks=yticks, show_legend=False, aspect=1.5, responsive=True,
    shared_axes=False, backend_opts={
        "x_range.bounds": (time.min(), time.max()),
        "y_range.bounds": (data.min(), max_data)})

# Get the y positions of the yticks to use as yaxis of minimap image
y_positions, _ = zip(*yticks)

# Compute z-scores across time for each channel
z_data = zscore(data, axis=1)

# Generate the zscored image for the minimap using the y tick positions from the eeg_viewer
minimap = hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)")
minimap = minimap.opts(
    cmap="RdBu_r", colorbar=False, xlabel='', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=100, responsive=True, default_tools=[''], shared_axes=False, clim=(-z_data.std()*2.5, z_data.std()*2.5))

# Create RangeToolLink between the minimap and the main EEG viewer 
if len(channels) < max_ch_disp:
    max_ch_disp = len(channels)
max_y_disp = np.max(data[max_ch_disp-1,:] + ((max_ch_disp-1) * offset))

time_s = len(time)/raw.info['sfreq']
if time_s < max_t_disp:
    max_t_disp = time_s
    
RangeToolLink(minimap, list(eeg_viewer.values())[0], axes=["x", "y"],
              boundsx=(None, max_t_disp),
              boundsy=(None, max_y_disp))

eeg_app = pn.Column(pn.Row(eeg_viewer, min_height=500), minimap).servable(target='main')
eeg_app

# Scratch

In [ ]:
# This dataset is ~20 MB
url = "https://github.com/mne-tools/mne-testing-data/raw/master/EDF/test_edf_stim_resamp.edf"
local_data_path = "../../data/"

# Will not download if already present at local_data_path
local_file_path = download_file(url, local_data_path)

In [ ]:
raw = mne.io.read_raw_edf(local_file_path)

In [ ]:
time = raw.times
channels = raw.ch_names

eeg_indices = mne.pick_types(raw.info, eeg=True)

eeg_indices

data = raw.get_data(picks=eeg_indices)

data.shape

t_crop_start = 0
t_crop_end = 60000
ch_crop_start = 50
ch_crop_end = 60
data = data[ch_crop_start:ch_crop_end,t_crop_start:t_crop_end]
time = time[t_crop_start:t_crop_end]
channels = channels[ch_crop_start:ch_crop_end]

In [ ]:
# # Set vertical spacing for EEG traces to avoid visual overlap
# spacing = 1.2
# offset = np.max(np.abs(data)) * spacing

# spacing = 2.0  # adjust this value to increase or decrease the spacing between traces
# offset = np.std(data) * spacing

# # Create a hv.Curve element per channel
# channel_curves = []
# max_data = data.max()
# for i, channel_data in enumerate(data):
#     offset_data = channel_data + (i * offset)
#     max_data = max(offset_data.max(), max_data)  # update max
#     channel_curves.append(
#         hv.Curve((time, offset_data), "Time").opts(
#             color="black", line_width=1,
#             tools=["hover", 'xwheel_zoom'], shared_axes=False))

# # Create mapping from yaxis location to ytick for each channel
# yticks = [(i * offset, ich) for i, ich in enumerate(channels)]

# # Create an overlay of curves
# eeg_viewer = hv.Overlay(channel_curves, kdims="Channel").opts(
#     padding=0, xlabel="Time (s)", ylabel="Channel", yticks=yticks, show_legend=False, aspect=1.5, responsive=True,
#     shared_axes=False, backend_opts={
#         "x_range.bounds": (time.min(), time.max()),
#         "y_range.bounds": (data.min(), max_data)})

# # Get the y positions of the yticks to use as yaxis of minimap image
# y_positions, _ = zip(*yticks)

# # Compute z-scores across time for each channel
# z_data = zscore(data, axis=1)

# # Generate the zscored image for the minimap using the y tick positions from the eeg_viewer
# minimap = hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)")

# # Style the minimap 
# minimap = minimap.opts(
#     cmap="RdBu_r", colorbar=False, xlabel='', alpha=.5, yticks=[yticks[0], yticks[-1]],
#     height=100, responsive=True, default_tools=[''], shared_axes=False, clim=(-z_data.std()*2.5, z_data.std()*2.5))

# # Create RangeToolLink between the minimap and the main EEG viewer 
# max_ch_disp = 10  # max channels to initially display
# if len(channels) < max_ch_disp:
#     max_ch_disp = len(channels)
# max_y_disp = np.max(data[max_ch_disp-1,:] + ((max_ch_disp-1) * offset))

# max_t_disp = 5
# time_s = len(time)/raw.info['sfreq']
# if time_s < max_t_disp:
#     max_t_disp = time_s
    
# RangeToolLink(minimap, list(eeg_viewer.values())[0], axes=["x", "y"],
#               boundsx=(None, max_t_disp),
#               boundsy=(None, max_y_disp))

# # Display vertically
# eeg_app = pn.Column(pn.Row(eeg_viewer, min_height=500), minimap).servable(target='main')
# eeg_app